In [1]:
from main import run_main
import json
import pandas as pd
import time
from copy import copy
from concurrent.futures import ThreadPoolExecutor
import threading

In [2]:
CROSSOVER_METHODS = ["one_point", "double_point", "uniform", "anular"]
SELECTION_METHODS = ["elite", "roulette", "universal", "boltzmann", "deterministic_tournament", "probabilistic_tournament", "ranking"]
MUTATION_METHODS = ["gene_mutation","uniform_multigen", "complete_mutation", "limited_multigen"]
GENES = ["items", "class", "height"]
NEW_GENERATION_METHODS = ["use_all", "new_over_actual"]
CHARACTER_TYPES = ["warrior", "archer", "defender", "infiltrator"]

In [3]:
config = {
    "N": 100, 
    "K": 60,
    "M": 30,
    "threshold": 0.7,
    "character_type": "archer",
    "selection_method_1": "universal",
    "selection_method_2": "universal", 
    "selection_method_3": "universal", 
    "selection_method_4": "universal",  
    "crossing_method": "one_point",
    "mutation_method": "uniform_multigen",
    "new_generation_method": "new_over_actual",
    "gene": "height",
    "A": 0.5,
    "B": 0.5,
    "p_m": 0.1,
    "Tc": 0.1,
    "T0": 10,
    "k": 2,
    "max_generations": 150,
    "max_generations_without_change": 10,
    "delta": 0.01
}

In [4]:


totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS) * 
                   len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
                   len(CHARACTER_TYPES) * 10)

currentIteration = 0
startTime = time.time()

df = []

for cross in CROSSOVER_METHODS:
    for sel1 in SELECTION_METHODS:
        for mut in MUTATION_METHODS:
            for gen in GENES:
                for new_gen in NEW_GENERATION_METHODS:
                    for clase in CHARACTER_TYPES:
                        for _ in range(10):
                            configCopy = copy(config)
                            configCopy["character_type"] = clase
                            configCopy["crossing_method"] = cross
                            configCopy["gene"] = gen
                            configCopy["mutation_method"] = mut
                            configCopy["new_generation_method"] = new_gen
                            configCopy["selection_method_1"] = sel1
                            configCopy["selection_method_2"] = sel1
                            configCopy["selection_method_3"] = sel1
                            configCopy["selection_method_4"] = sel1

                            df.append((run_main(configCopy), configCopy))

                            currentIteration += 1
                            if currentIteration % 100 == 0:  # Actualizar cada 100 iteraciones
                                elapsedTime = time.time() - startTime
                                estimatedRemainingTime = (totalIterations - currentIteration) * (elapsedTime / currentIteration)
                                percentageCompletion = (currentIteration / totalIterations) * 100
                                print(f"\rCompletion: {percentageCompletion:.2f}% ETA: {estimatedRemainingTime:.2f}s", end="")

# Crear un bloqueo global para evitar condiciones de carrera

# totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS) * 
#                    len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
#                    len(CHARACTER_TYPES) * 10)

# df = []
# startTime = time.time()

# currentIteration = 0

# lock = threading.Lock()

# def runInThread(cross, sel1, mut, gen, new_gen, clase, config):
#     # global currentIteration  # Hacer que la variable sea global para que pueda ser modificada

#     configCopy = copy(config)
#     configCopy["character_type"] = clase
#     configCopy["crossing_method"] = cross
#     configCopy["gene"] = gen
#     configCopy["mutation_method"] = mut
#     configCopy["new_generation_method"] = new_gen
#     configCopy["selection_method_1"] = sel1
#     configCopy["selection_method_2"] = sel1
#     configCopy["selection_method_3"] = sel1
#     configCopy["selection_method_4"] = sel1

#     result = (run_main(configCopy), configCopy)
    
#     # Adquirir bloqueo antes de modificar variables compartidas
#     with lock:
#         df.append(result)
#         currentIteration += 1
#         if currentIteration % 50 == 0:
#             elapsedTime = time.time() - startTime
#             estimatedRemainingTime = (totalIterations - currentIteration) * (elapsedTime / currentIteration)
#             percentageCompletion = (currentIteration / totalIterations) * 100
#             print(f"\rCompletion: {percentageCompletion:.2f}% ETA: {estimatedRemainingTime:.2f}s", end="")

# # Uso de ThreadPoolExecutor para manejar threads
# with ThreadPoolExecutor(max_workers=2) as executor:
#     for cross in CROSSOVER_METHODS:
#         for sel1 in SELECTION_METHODS:
#             for mut in MUTATION_METHODS:
#                 for gen in GENES:
#                     for new_gen in NEW_GENERATION_METHODS:
#                         for clase in CHARACTER_TYPES:
#                             for _ in range(10):
#                                 executor.submit(runInThread, cross, sel1, mut, gen, new_gen, clase, config)

Completion: 99.70% ETA: 94.50ssss

In [5]:
df2 = [{**{"strength": char[0]._items["strength"],
           "agility": char[0]._items["agility"],
           "expertise": char[0]._items["expertise"],
           "endurance": char[0]._items["endurance"],
           "health": char[0]._items["health"],
           "height": char[0]._height,
           "performance": char[0].get_performance(),
           "final_character_type":char[0]._class,
           "generation": char[1]},
        **conf}
       for char, conf in df]

df2 = pd.DataFrame(df2)
df2

,strength,agility,expertise,endurance,health,height,performance,final_character_type,generation,N,...,gene,A,B,p_m,Tc,T0,k,max_generations,max_generations_without_change,delta
0,45.300728,43.468828,27.996254,13.539494,19.694696,1.944433,26.465618,<class 'character_class.Warrior'>,35,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
1,47.504664,39.094945,30.459878,14.163975,18.776538,1.838464,25.629465,<class 'character_class.Warrior'>,31,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
2,44.775352,38.515830,30.690703,20.286389,15.731726,1.906934,25.308789,<class 'character_class.Warrior'>,22,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
3,43.283387,40.778603,28.121785,15.833437,21.982787,1.912724,25.185196,<class 'character_class.Warrior'>,28,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
4,45.765046,34.469087,29.887925,16.688781,23.189161,1.951162,24.303052,<class 'character_class.Warrior'>,24,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26875,49.991151,37.354456,25.585374,17.721274,19.347746,1.919915,36.444890,<class 'character_class.Archer'>,66,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26876,41.011928,35.600166,30.749730,22.053097,20.585079,1.902412,31.613506,<class 'character_class.Archer'>,48,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26877,48.938115,34.829953,30.740919,19.980986,15.510028,1.915617,36.216022,<class 'character_class.Archer'>,43,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26878,49.396989,36.538088,27.966473,18.812489,17.285961,1.896510,36.363286,<class 'character_class.Archer'>,50,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01


In [6]:
df2.to_csv("datos.csv", index=False)

In [19]:
df2

,strength,agility,expertise,endurance,health,height,performance,final_character_type,generation,N,...,gene,A,B,p_m,Tc,T0,k,max_generations,max_generations_without_change,delta
0,45.300728,43.468828,27.996254,13.539494,19.694696,1.944433,26.465618,<class 'character_class.Warrior'>,35,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
1,47.504664,39.094945,30.459878,14.163975,18.776538,1.838464,25.629465,<class 'character_class.Warrior'>,31,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
2,44.775352,38.515830,30.690703,20.286389,15.731726,1.906934,25.308789,<class 'character_class.Warrior'>,22,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
3,43.283387,40.778603,28.121785,15.833437,21.982787,1.912724,25.185196,<class 'character_class.Warrior'>,28,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
4,45.765046,34.469087,29.887925,16.688781,23.189161,1.951162,24.303052,<class 'character_class.Warrior'>,24,100,...,items,0.5,0.5,0.1,0.1,10,2,150,10,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26875,49.991151,37.354456,25.585374,17.721274,19.347746,1.919915,36.444890,<class 'character_class.Archer'>,66,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26876,41.011928,35.600166,30.749730,22.053097,20.585079,1.902412,31.613506,<class 'character_class.Archer'>,48,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26877,48.938115,34.829953,30.740919,19.980986,15.510028,1.915617,36.216022,<class 'character_class.Archer'>,43,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01
26878,49.396989,36.538088,27.966473,18.812489,17.285961,1.896510,36.363286,<class 'character_class.Archer'>,50,100,...,height,0.5,0.5,0.1,0.1,10,2,150,10,0.01


Para poder hacer un analisis del crossing se necesitan mas datos sobre la combinacion de metodos de seleccion

In [ ]:
config = {
    "N": 10, 
    "K": 6,
    "M": 3,
    "threshold": 0.7,
    "character_type": "archer",
    "selection_method_1": "universal",
    "selection_method_2": "universal", 
    "selection_method_3": "universal", 
    "selection_method_4": "universal",  
    "crossing_method": "one_point",
    "mutation_method": "uniform_multigen",
    "new_generation_method": "new_over_actual",
    "gene": "height",
    "A": 0.5,
    "B": 0.5,
    "p_m": 0.1,
    "Tc": 0.1,
    "T0": 10,
    "k": 2,
    "max_generations": 100,
    "max_generations_without_change": 5,
    "delta": 0.01
}

In [5]:

totalIterations = (len(CROSSOVER_METHODS) * len(SELECTION_METHODS)**4 * 
                   len(MUTATION_METHODS) * len(GENES) * len(NEW_GENERATION_METHODS) * 
                   len(CHARACTER_TYPES) * 3)

currentIteration = 0
startTime = time.time()

df_aux = []

for cross in CROSSOVER_METHODS:
    for sel1 in SELECTION_METHODS:
        for sel2 in SELECTION_METHODS:
            for sel3 in SELECTION_METHODS:
                for sel4 in SELECTION_METHODS:
                    for mut in MUTATION_METHODS:
                        for gen in GENES:
                            for new_gen in NEW_GENERATION_METHODS:
                                for clase in CHARACTER_TYPES:
                                    for _ in range(3):
                                        configCopy = copy(config)
                                        configCopy["character_type"] = clase
                                        configCopy["crossing_method"] = cross
                                        configCopy["gene"] = gen
                                        configCopy["mutation_method"] = mut
                                        configCopy["new_generation_method"] = new_gen
                                        configCopy["selection_method_1"] = sel1
                                        configCopy["selection_method_2"] = sel2
                                        configCopy["selection_method_3"] = sel3
                                        configCopy["selection_method_4"] = sel4

                                        df_aux.append((run_main(configCopy), configCopy))

                                        currentIteration += 1
                                        if currentIteration % 100 == 0:  # Actualizar cada 100 iteraciones
                                            elapsedTime = time.time() - startTime
                                            estimatedRemainingTime = (totalIterations - currentIteration) * (elapsedTime / currentIteration)
                                            percentageCompletion = (currentIteration / totalIterations) * 100
                                            print(f"\rCompletion: {percentageCompletion:.2f}% ETA: {estimatedRemainingTime:.2f}s", end="")

Completion: 45.44% ETA: 11649.58s

In [ ]:
df_aux2 = [{**{"strength": char[0]._items["strength"],
           "agility": char[0]._items["agility"],
           "expertise": char[0]._items["expertise"],
           "endurance": char[0]._items["endurance"],
           "health": char[0]._items["health"],
           "height": char[0]._height,
           "performance": char[0].get_performance(),
           "final_character_type":char[0]._class,
           "generation": char[1]},
        **conf}
       for char, conf in df_aux]

df_aux2 = pd.DataFrame(df_aux2)
df_aux2

In [ ]:
df_aux2.to_csv("datos_cruce.csv", index=False)